# SqlModel

doc: https://sqlmodel.tiangolo.com/


# 定义表


In [ ]:
from typing import Optional

from sqlmodel import Field, SQLModel


class Hero(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    secret_name: str
    age: Optional[int] = None
    # money: str = Field(index=True)   带索引的字段

# 创建表


In [ ]:
from sqlmodel import SQLModel, create_engine

# echo=True 代表打印sql细节
engine = create_engine("sqlite:///database.db", echo=False)

In [ ]:
# 创建方式1
SQLModel.metadata.create_all(engine)

In [ ]:
# 创建方式2
# checkfirst表示创建表前先检查该表是否存在,已存在则不再创建(默认True)
Hero.__table__.create(engine, checkfirst=True)

# 插入


In [ ]:
from sqlmodel import Session


hero_1 = Hero(name="Deadpond", secret_name="Dive Wilson")
hero_2 = Hero(name="Spider-Boy", secret_name="Pedro Parqueador")
hero_3 = Hero(name="Rusty-Man", secret_name="Tommy Sharp", age=48)

with Session(engine) as session:
    session.add(hero_1)
    session.add(hero_2)
    session.add(hero_3)
    session.commit()

# 普通查询


## 查询全部

- exec 查询返回的是 sqlalchemy.engine.result.ScalarResul
- sqlmodel 其实是一个封装 sqlalchemy 的简化框架


In [17]:
from sqlmodel import Session, select

with Session(engine) as session:
    statement = select(Hero)
    results = session.exec(statement)

    print(type(results))
    for hero in results:
        print(type(hero))
        print(hero)

<class 'sqlalchemy.engine.result.ScalarResult'>
<class '__main__.Hero'>
name='Deadpond' secret_name='Dive Wilson' id=1 age=None
<class '__main__.Hero'>
name='Spider-Boy' secret_name='Pedro Parqueador' id=2 age=None
<class '__main__.Hero'>
name='Rusty-Man' secret_name='Tommy Sharp' id=3 age=48


## 查询 list


In [11]:
with Session(engine) as session:
    statement = select(Hero)
    results = session.exec(statement)
    heroes = results.all()

    print(type(heroes))
    print(heroes)

<class 'list'>
[Hero(name='Deadpond', secret_name='Dive Wilson', id=1, age=None), Hero(name='Spider-Boy', secret_name='Pedro Parqueador', id=2, age=None), Hero(name='Rusty-Man', secret_name='Tommy Sharp', id=3, age=48)]


## 查询一条


In [36]:
from sqlmodel import Session, select

with Session(engine) as session:
    statement = select(Hero).where(Hero.name == "Spider-Boy")
    hero = session.exec(statement).first()
    print(hero)

name='Spider-Boy' secret_name='Pedro Parqueador' id=2 age=None


### where 条件的本质

Model.field 是一个 sqlalchemy 表达式对象


In [23]:
Hero.name == "Spider-Boy"

### 主键查询

使用 session.get()就代表根据表的主键查询


In [33]:
with Session(engine) as session:
    results = session.get(Hero, 1)
    print("Hero:", results)

Hero: name='Deadpond' secret_name='Dive Wilson' id=1 age=None


In [35]:
# 如果主键不存在 返回None
with Session(engine) as session:
    results = session.get(Hero, 4)
    print("Hero:", results)

Hero: None


### 查询结果空-返回 None

使用 first()


In [27]:
with Session(engine) as session:
    statement = select(Hero).where(Hero.age < 25)
    results = session.exec(statement)
    hero = results.first()
    print("Hero:", hero)

Hero: None


### 查询结果为空-报错

使用 one()


In [30]:
try:
    with Session(engine) as session:
        statement = select(Hero).where(Hero.age < 25)
        results = session.exec(statement)
        hero = results.one()
        print("Hero:", hero)
except Exception as e:
    print(e)
    print(type(e))

No row was found when one was required
<class 'sqlalchemy.exc.NoResultFound'>


## 范围查询


In [13]:
with Session(engine) as session:
    statement = select(Hero).where(Hero.age > 35)
    results = session.exec(statement)
    for hero in results:
        print(hero)

name='Rusty-Man' secret_name='Tommy Sharp' id=3 age=48


## 多个查询条件


In [19]:
with Session(engine) as session:
    statement = select(Hero).where(Hero.age >= 35).where(Hero.name == "Rusty-Man")
    results = session.exec(statement)
    for hero in results:
        print(hero)

name='Rusty-Man' secret_name='Tommy Sharp' id=3 age=48


## or 查询


In [22]:
from sqlmodel import or_

with Session(engine) as session:
    statement = select(Hero).where(or_(Hero.age <= 35, Hero.name == "Rusty-Man"))
    results = session.exec(statement)
    for hero in results:
        print(hero)

name='Rusty-Man' secret_name='Tommy Sharp' id=3 age=48


## limit 查询


In [37]:
with Session(engine) as session:
    statement = select(Hero).limit(2)
    results = session.exec(statement)
    for hero in results:
        print(hero)

name='Deadpond' secret_name='Dive Wilson' id=1 age=None
name='Spider-Boy' secret_name='Pedro Parqueador' id=2 age=None


In [39]:
# 分页查询，每页2条，第2页
with Session(engine) as session:
    statement = select(Hero).limit(2).offset(2)
    results = session.exec(statement)
    for hero in results:
        print(hero)

name='Rusty-Man' secret_name='Tommy Sharp' id=3 age=48


## 排序


In [52]:
with Session(engine) as session:
    statement = select(Hero).order_by(Hero.age.desc())
    results = session.exec(statement)
    for hero in results:
        print(hero)

name='Rusty-Man' secret_name='Tommy Sharp' id=3 age=48
name='Deadpond' secret_name='Dive Wilson' id=1 age=None
name='Spider-Boy' secret_name='Pedro Parqueador' id=2 age=None


## count


In [64]:
from sqlalchemy import func

with Session(engine) as session:
    statement = select(func.count(Hero.id))
    result = session.exec(statement).first()
    print(result)

3


## group by


In [79]:
with Session(engine) as session:
    statement = select(Hero.age, func.count(Hero.name)).group_by(Hero.age)
    results = session.exec(statement).all()
    for hero in results:
        print(type(hero))
        print(hero)

<class 'sqlalchemy.engine.row.Row'>
(None, 2)
<class 'sqlalchemy.engine.row.Row'>
(48, 1)


# 更新


In [81]:
with Session(engine) as session:
    hero = session.get(Hero, 1)
    print(hero)
    hero.age = 50
    session.add(hero)
    session.commit()

    print(session.get(Hero, 1))

name='Deadpond' secret_name='Dive Wilson' id=1 age=None
id=1 name='Deadpond' secret_name='Dive Wilson' age=50


# 删除


In [82]:
with Session(engine) as session:
    session.delete(session.get(Hero, 1))
    session.commit()
    print(session.get(Hero, 1))

None
